### 第一步通过三元组构建知识图谱

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from collections import defaultdict
import os
import json

# 加载多语言句子嵌入模型,用于生成句子的嵌入向量，方便后续的相似度检查
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  

DATA_PATH = './data/harry.json'

# 第一部分构建Graph类
class Knowledgebase:
    def __init__(self,data_path=DATA_PATH):
        self.graph = defaultdict(list)
        self.nodes = set()
        self.embeddings = {}
        self.node_attributes = defaultdict(dict)
        self.community_labels = {} # 可以在后续进行社区检测完后填充
        with open(data_path, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
        self._build_graph_from_json(raw_data)

        # 生成节点向量，这样可以基于相似度优化查询过程
        node_list = list(self.nodes)
        embeddings = model.encode(node_list, show_progress_bar=True)
        for node, emb in zip(node_list,embeddings):
            self.embeddings[node] = emb


    def _build_graph_from_json(self,raw_data):
        '''
        将json解析为图结构，同时添加反向边以构建无向图（或双向图）
        '''
        # key 是人物名称，value是属性字典
        for name, info in raw_data.items():
            self.nodes.add(name)
            # 然后对属性进行处理
            attributes = ['出生', '血统', '婚姻状况', '物种', '性别', '职业', '从属']
            for attr in attributes:
                if attr in info:
                    # 在节点属性中存放各种属性
                    self.node_attributes[name][attr] = info[attr]
            
            # 处理家庭信息，建立图的边的关系
            if '家庭信息' in info:
                for relation, relatives in info['家庭信息'].items():
                    if isinstance(relatives, str):
                        relatives = [relatives]
                    for relative in relatives:
                        # 正向边: name -> relative
                        self.graph[name].append((relation, relative))
                        # 反向边: relative -> name (添加前缀 ~ 表示反向)
                        self.graph[relative].append(("~" + relation, name))
                        
                        self.nodes.add(relative)
            
            # 处理职业关系、从属关系
            for key in ['职业','从属']:
                if key in info:
                    relations = info[key]
                    if isinstance(relations, str):
                        relations = [relations]
                    for relation in relations:
                        # 正向边
                        self.graph[name].append((key, relation))
                        # 反向边 (例如: 霍格沃茨 -> 哈利)
                        self.graph[relation].append(("~" + key, name))
                        
                        
                        self.nodes.add(relation)        
    
    def get_neighbors(self, node):
        return self.graph.get(node, [])

    def get_embedding(self, node):
        return self.embeddings.get(node, None)

    def get_all_nodes(self):
        return list(self.nodes)
    
    def get_undirected_neighbors(self):
        # 获取无向图 (用于社区发现)
        # 由于我们现在 graph 本身就是双向的，这里逻辑依然适用，只是会有重复边，set会自动去重
        undirected = defaultdict(set)
        for head,edges in self.graph.items():
            for relation,tail in edges:
                undirected[head].add(tail)
                undirected[tail].add(head)
        return undirected
    
    def _compute_degrees(self,undirected):
        degrees = {}
        for node, neighbors in undirected.items():
            degrees[node] = len(neighbors)
        return degrees

In [129]:
# --- 测试部分 ---
DATA_PATH = './data/harry.json'
if __name__ == "__main__":
    kb = Knowledgebase(DATA_PATH)
    print(f"\n✅ 中文图谱构建成功！")
    print(f"总节点数: {len(kb.nodes)}")
    
    test_node = "乔治·韦斯莱"
    if test_node in kb.nodes:
        print(f"\n[{test_node}] 的关系网示例:")
        # 打印前5个邻居
        for neighbor, relation in kb.get_neighbors(test_node)[:5]:
            print(f"  --[{relation}]--> {neighbor}")


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Batches: 100%|██████████| 45/45 [00:00<00:00, 140.26it/s]


✅ 中文图谱构建成功！
总节点数: 1437

[乔治·韦斯莱] 的关系网示例:
  --[罗恩·韦斯莱]--> ~哥哥
  --[塞普蒂默斯·韦斯莱]--> 祖父
  --[塞德瑞拉·布莱克]--> 祖母
  --[普威特先生]--> 外祖父
  --[普威特夫人]--> 外祖母


### 通过LPA算法进行社区划分
在第一版中，我采取了以下步骤：
1. 初始化每个节点的标签为其自身。
2. 迭代更新每个节点的标签为其邻居中出现频率最高的标签。
3. 重复上述过程直到标签不再变化或达到最大迭代次数。

但是该方法处理结果并不理想，社区划分效果较差。最后发现应该是数据集的原因，换了数据集发现效果还不错。

In [117]:
# LPA算法实现社区发现
# 初始化标签，每个节点的标签初始为其自身
def init_labels(nodes):
    labels= {node: node for node in nodes}
    return labels


# LPA算法核心步骤： 投票更新标签
def LPA_step(node,graph, labels):
    '''
    输入：
        node: 当前要更新的节点
        graph： 图
        labels: 当前节点标签字典
    '''
    neighbors = graph.get(node, [])
    if not neighbors:
        return labels[node] #  没有邻居，标签不变

    label_count = defaultdict(int)
    for relation,neighbor in neighbors:
        neighbor_label = labels.get(neighbor, None)
        label_count[neighbor_label] += 1

    # 选择出现频率最高的标签
    max_count =max(label_count.values())
    best_labels = [label for label, count in label_count.items() if count == max_count]

    # 如果有多个标签频率相同，随机选择一个
    new_label = np.random.choice(best_labels)
    return new_label

# 运行LPA算法直到收敛或者最大迭代次数
def LPA(graph, max_iterations=100,all_nodes=None):
    # 初始化标签 {node: label}
    # 修正点1：初始化时应该使用 all_nodes，而不是 graph.keys()
    # graph.keys() 只包含作为起点的节点，可能会漏掉只作为终点的节点
    labels = init_labels(all_nodes)
    
    for iteration in range(max_iterations):
        prev_labels = labels.copy()
        
        #打乱节点更新顺序
        # LPA 算法对更新顺序敏感，随机打乱可以避免陷入局部最优，并加速收敛
        np.random.shuffle(all_nodes)
        
        for node in all_nodes:
            labels[node] = LPA_step(node, graph, labels)
        # 检查是否收敛
        if labels == prev_labels:
            print(f"算法在第 {iteration} 次迭代后收敛。")
            break
    return labels

### SLPA 算法改进
SLPA算法是对LPA算法的改进，由于在作业过程中，LPA算法和Louvain算法都是每个节点只能属于一个社区，但是对于当前数据集复杂的人物关系，这种划分会导致有些关系密切的反而检查不到，比如同属于一个学院的两个人。

In [ ]:
# SLPA (Speaker-Listener Label Propagation Algorithm) 实现重叠社区发现
def SLPA(graph, T=20, threshold=0.1, all_nodes=None):
    '''
    SLPA 算法：允许重叠社区发现
    :param graph: 图结构
    :param T: 迭代次数
    :param threshold: 概率阈值，超过该概率的标签会被保留
    :param all_nodes: 所有节点列表
    '''
    # 每个节点记忆的标签中最开始只包含自己
    memory = {node: [node] for node in all_nodes}
    
    for t in range(T):
        # 随机顺序遍历节点（作为 Listener）
        np.random.shuffle(all_nodes)
        
        for listener in all_nodes:
            neighbors = graph.get(listener, [])
            if not neighbors:
                continue
            
            #  每个邻居 (Speaker) 随机从自己的内存中说出一个标签
            spoken_labels = []
            for relation, speaker in neighbors:
                if speaker in memory:
                    speaker_mem = memory[speaker]
                    # 按照概率随机选择一个标签（简单起见，这里直接均匀随机选择，标准SLPA是按频率采样）
                    label = np.random.choice(speaker_mem)
                    spoken_labels.append(label)
            
            if not spoken_labels:
                continue
                
            # 接受出现次数最多的标签，加入自己的内存
            label_counts = defaultdict(int)
            for label in spoken_labels:
                label_counts[label] += 1
            
            max_freq = max(label_counts.values())
            # 可能有多个标签频率并列第一
            best_labels = [l for l, c in label_counts.items() if c == max_freq]
            selected_label = np.random.choice(best_labels)
            
            memory[listener].append(selected_label)
            
    # 根据阈值生成社区标签集合
    communities = {}
    for node, mem in memory.items():
        total = len(mem)
        counts = defaultdict(int)
        for label in mem:
            counts[label] += 1
        
        # 保留出现概率 >= threshold 的标签
        node_communities = set()
        for label, count in counts.items():
            if count / total >= threshold:
                node_communities.add(label)
        
        communities[node] = node_communities
                
    return communities

In [128]:
# 测试部分
if  __name__ == '__main__':
    kb = Knowledgebase(DATA_PATH)
    all_nodes = kb.get_all_nodes()
    community_labels = LPA(kb.graph, max_iterations=1000,all_nodes = all_nodes)

    # 输出部分社区划分结果
    from collections import defaultdict
    communities = defaultdict(list)
    for node, label in community_labels.items():
        communities[label].append(node)

    print(f"\n发现的社区数量: {len(communities)}")
    cnt = 0
    # 为了展示结果，我们这里选取成员数目较多的社区进行打印
    for i, (label, members) in enumerate(communities.items()):
        
        if cnt >= 4:  
            break

        if len(members) > 20:
            print(f"\n社区 {i+1} (标签: {label}) 包含成员:")
            for member in members:
                print(f" - {member}")
            print(len(members))
            cnt += 1
        else:
            continue

Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Batches: 100%|██████████| 45/45 [00:00<00:00, 118.95it/s]



发现的社区数量: 250

社区 2 (标签: 查理·韦斯莱) 包含成员:
 - 查德里火炮队
 - 布斯巴顿魔法学院
 - 魔法生物管理控制司 (曾经)
 - 《预言家日报》魁地奇资深记者
 - 亚瑟·韦斯莱
 - 伊万斯姐妹的父亲
 - 泰丝
 - 莫丽·普威特
 - 古灵阁巫师银行
 - 乔艾·詹肯斯
 - 禁止滥用麻瓜物品办公室主任
 - 哈德温·波特
 - 费比安·普威特
 - 弗雷德·韦斯莱
 - 赫敏·格兰杰
 - 罗恩·韦斯莱
 - 珀克斯家族
 - 赫敏·韦斯莱
 - 拉环
 - 威森加摩成员 (1612年-1652年)
 - 莉莉·伊万斯
 - 先遣警卫
 - 伊格内修斯·普威特
 - 雨果·韦斯莱
 - 路易·韦斯莱
 - 佩弗利尔
 - 他的同伙
 - 德思礼夫人
 - 麦金农家庭
 - 阿不思·波特
 - 哈利·波特
 - 凤凰社
 - 在古灵阁巫师银行的兼职
 - 莉莉·波特
 - 德拉戈米尔·高尔格维奇
 - 乔治·韦斯莱
 - 珀西·韦斯莱
 - 伊万斯家庭
 - 查德里火炮队追球手
 - 格兰芬多学院
 - 弗雷德·韦斯莱二世
 - 波特夫人
 - 比尔·韦斯莱
 - 弗利蒙家族
 - 普威特先生
 - 波特家族
 - 弗利蒙·波特
 - 禁止滥用麻瓜物品办公室
 - 卢克丽霞·布莱克
 - 罗克珊·韦斯莱
 - 死亡圣器
 - 查理·韦斯莱
 - 弗农·德思礼
 - 佩弗利尔家族
 - 劫盗者
 - 奥黛丽·韦斯莱
 - 黑魔法防御术兼职讲师
 - 韦斯莱魔法把戏坊
 - 尤菲米娅·波特
 - 麦克米兰家族
 - 阿波琳·德拉库尔
 - 塞德瑞拉·布莱克
 - 伪劣防御咒及防护用品侦查收缴办公室
 - 格朗宁公司主管
 - 厄尼·麦克米兰
 - 斯廷奇库姆的林弗雷德
 - 韦斯莱家的孩子
 - 亨利·波特的母亲
 - 莉莉·卢娜·波特
 - 魔法法律执行司副司长
 - 梅林达·波宾
 - 詹姆·波特
 - 梅拉尼娅·麦克米兰
 - 莫丽·韦斯莱
 - 格朗宁公司
 - 罗丝·韦斯莱
 - 傲罗 (退休)
 - 阿拉斯托·穆迪
 - 古灵阁巫师银行雇员 (曾经)
 - 安德鲁·柯克
 - 格兰莫·珀克斯
 - 哈维·瑞吉比特
 - 佩妮·德思礼
 - 牛头犬饲养员
 - 路易斯·韦斯莱
 - 霍利黑德哈比队魁地奇球员
 - 吉米·

### 换另一种基于贪心策略的算法：Louvain算法
参考资料：
- https://zhuanlan.zhihu.com/p/556291759?s_r=0
- Google AI studio 生成的公式推导

介绍： Louvain算法是一种用于社区检测的贪心优化算法，旨在最大化网络的模块度（Modularity）。模块度是衡量网络划分质量的指标，反映了网络中节点之间连接的密集程度。Louvain算法通过迭代地合并节点和社区来提高模块度，从而发现网络中的社区结构。

主要步骤：
1. **初始化**：将每个节点视为一个独立的社区。
2. **局部优化**：对于每个节点，计算将其移动到邻居社区所带来的模块度增益。如果移动能够提高模块度，则将节点移动到该社区。重复此过程，直到没有节点能够通过移动来提高模块度。
3. **社区合并**：将每个社区视为一个节点，构建新的网络。重复步骤2和3，直到模块度不再显著增加。

Q：模块度定义公式如下：

$$
Q = \frac{1}{2m} \sum_{i,j} \left[ A_{ij} - \frac{k_i k_j}{2m} \right] \delta(c_i, c_j)
$$

其中：

- $ Q $ 是模块度。
- $ m $ 是图中的总边数。
- $ A_{ij} $ 是节点 $ i $ 和节点 $ j $ 之间的边的权重（对于无权图，若存在边则为 1，否则为 0）。
- $ k_i $ 和 $ k_j $ 分别是节点 $ i $ 和节点 $ j $ 的度数。
- $ c_i $ 和 $ c_j $ 分别是节点 $ i $ 和节点 $ j $ 所属的社区。
- $ \delta(c_i, c_j) $ 是一个指示函数，当 $ c_i = c_j $ 时取值为 1，否则为 0，即只有当两个节点属于同一个社区时才计入计算。

相较于LPA算法，Louvain算法更难理解一点。
这里查资料详解一下Louvain算法中模块度衡量的原理，以及delta_Q公式的优化过程：

(1) **核心公式**：
  $$ P_{ij} = \frac{k_i k_j}{2m} $$
  该值表示在随机网络中，节点 $i$ 和节点 $j$ 之间产生连接的**期望边数**（或期望权重）。

- **推导过程（直观理解）**：（考完研之后这些全忘了，所以让AI讲解了一下，但是我还是没有完全get到）
  1.  **定义线头（Stubs）**：
      - 网络中总共有 $m$ 条边，如果把边切断，总共有 $2m$ 个线头（Stub）。
      - 节点 $i$ 拥有 $k_i$ 个线头，节点 $j$ 拥有 $k_j$ 个线头。
  2.  **随机匹配概率**：
      - 想象我们随机抓取一个线头，它属于节点 $i$ 的概率是 $\frac{k_i}{2m}$。
      - 再随机抓取另一个线头，它属于节点 $j$ 的概率是 $\frac{k_j}{2m}$。
  3.  **计算期望值**：
      - 假设这两个选择是独立的，那么一条随机边连接 $i$ 和 $j$ 的联合概率约为 $\frac{k_i}{2m} \times \frac{k_j}{2m}$。
      - 因为网络中总共有 $2m$ 个端点（或者理解为有 $m$ 条边在寻找连接，且涉及方向性时的系数抵消），其数学期望（Expected Number of Edges）计算为：
        $$ E_{ij} = 2m \times \frac{k_i}{2m} \times \frac{k_j}{2m} = \frac{k_i k_j}{2m} $$

- **模块度 Q 的物理含义**：
  公式中的项 $$ A_{ij} - \frac{k_i k_j}{2m} $$ 代表了 **“实际连接”** 与 **“随机连接”** 之间的差值：
  - **$A_{ij}$**：节点 $i$ 和 $j$ 之间的实际边权重。
  - **$\frac{k_i k_j}{2m}$**：如果网络是随机连接的，它们之间理应具有的边权重。
  
  **结论**：如果差值为**正**，说明这两个节点之间的连接比随机猜测的要紧密得多，因此将它们划分到同一个社区是对模块度有益的。这体现了模块度的核心思想：**社区内的连接密度应显著高于随机网络的连接密度。**

（2）**模块度增量 $\Delta Q$ 的计算**：
在Louvain算法中，当考虑将节点 $i$ 从其当前社区$A$移动到邻居社区 $C$ 时，模块度的变化 $\Delta Q$ 可以通过以下公式计算：
$$
\Delta Q = \left[ \frac{\sum_{in} + k_{i,in}}{2m} - \left( \frac{\sum_{tot} + k_i}{2m} \right)^2 \right] - \left[ \frac{\sum_{in}}{2m} - \left( \frac{\sum_{tot}}{2m} \right)^2 - \left( \frac{k_i}{2m} \right)^2 \right]
$$
参数解释：
- $\sum_{in}$：社区 $C$ 内现有的边权重总和（无权图时为社区内边的数量）。
- $\sum_{tot}$：社区 $C$ 内所有节点的度数总和
- $k_{i,in}$：节点 $i$ 与社区 $C$ 内节点之间的边权重总和，无权图时为节点 $i$ 与社区内节点相连边的数量。
- $k_i$：节点 $i$ 的度数。  
该公式衡量了将节点 $i$ 移动到社区 $C$ 后，模块度 $Q$ 的变化。通过计算 $\Delta Q$，Louvain算法能够决定是否将节点 $i$ 移动到社区 $C$ 以最大化整体的模块度。

关于这部分还有一点，可能大家会有和我一样的疑惑，就是上述公式实际上只有$Q_{gain}$部分，公式第二部中$- \left( \frac{k_i}{2m} \right)^2$是$i$作为独立社区时的模块度贡献。
那么节点离开原来社区的模块损失度为什么不计算呢？
这是因为在Louvain算法的迭代过程中，节点 $i$ 的移动是基于当前社区划分的局部优化。也就是说，当我们计算将节点 $i$ 从其当前社区移动到邻居社区 $C$ 时，原社区的模块度是固定的，不会因为节点 $i$ 的移动而改变。因此，在计算模块度增量 $\Delta Q$ 时，我们只需要关注节点 $i$ 移动到社区 $C$ 后对模块度的影响，而不需要考虑它离开原社区所带来的模块度损失。

In [120]:
def louvain(kb,max_passes=1):
    '''
    手写Louvain算法实现社区发现
    '''
    # 无向图
    undirected = kb.get_undirected_neighbors()
    degrees = kb._compute_degrees(undirected)
    m = sum(degrees.values()) /2  # 图中的总边数
    
    # 初始化每个节点为一个社区,直接用节点名作为社区标签
    communities = {node: node for node in kb.get_all_nodes()} 
    node_list = kb.get_all_nodes() # list 类型

    # 贪心算法
    for _ in range(max_passes):
        improved = False
        for node in node_list:
            
            # 当前社区
            current_community = communities[node]

            # 计算当前节点和每个邻居社区的连接边数
            neighbor_communities = defaultdict(int)
            for neighbor in undirected[node]:
                neighbor_community = communities[neighbor]
                neighbor_communities[neighbor_community] += 1
            
            '''
            neighbor_communities 结构示例:
            {'社区A': 3,  # 当前节点与社区A有3条连接边}
            '''

            # 计算将节点移动到每个邻居社区的模块度增益
            best_increase = 0
            best_community = current_community

            # 当前社区的度
            # 修正：使用 .get() 方法防止 KeyError
            # 在迭代过程中，某些节点可能被移动到了新的社区，而 degrees 字典只包含原始节点的度数
            # 但这里的逻辑是计算社区的总度数，所以我们需要遍历所有属于该社区的节点
            degree_current = sum(degrees.get(n, 0) for n in communities if communities[n] == current_community)
            
            for community, k_i_in in neighbor_communities.items():
                if community == current_community:
                    continue
                
                # 可能进入的社区的度
                degree_community = sum(degrees.get(n, 0) for n in communities if communities[n] == community)
                # 计算模块增益
                delta_Q = (k_i_in / (2 * m))- (degrees[node] * degree_community) / (2 * m * m)
                
                if delta_Q > best_increase:
                    best_increase = delta_Q
                    best_community = community
            
            # 如果delta_Q > 0，则将节点移动到最佳社区
            if best_increase > 0 and best_community != current_community:
                communities[node] = best_community
                improved = True
        if not improved:
            break
    return communities

In [121]:
# 测试部分

if __name__ == '__main__':
    kb = Knowledgebase(DATA_PATH)
    community_labels = louvain(kb, max_passes= 4)

    # 输出部分社区划分结果
    from collections import defaultdict
    communities = defaultdict(list)
    for node, label in community_labels.items():
        communities[label].append(node)

    print(f"\nLouvain算法发现的社区数量: {len(communities)}")
    cnt = 0
    # 为了展示结果，我们这里选取成员数目较多的社区进行打印
    for i, (label, members) in enumerate(communities.items()):
        
        if cnt >= 4:  
            break

        if len(members) > 20:
            print(f"\n社区 {i+1} (标签: {label}) 包含成员:")
            for member in members:
                print(f" - {member}")
            print(len(members))
            cnt += 1
        else:
            continue
# 可视化测试


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Batches: 100%|██████████| 45/45 [00:00<00:00, 146.06it/s]



Louvain算法发现的社区数量: 276

社区 1 (标签: 神秘事务司) 包含成员:
 - 
 - 魔法部高级副部长 (1995年之前－1998年)
 - 马法尔达·霍普柯克
 - 魔法部官员
 - 奥福德·乌姆里奇
 - 威尔克斯
 - 斯坦·桑帕克
 - 霍格沃茨黑魔法防御术教授 (1995年－1996年)
 - 伏地魔
 - 威森加摩
 - 骑士公共汽车售票员
 - 麻瓜出身登记委员会
 - 缄默人 (曾经)
 - 高级调查官 of 霍格沃茨 (1995年－1996年)
 - 麻瓜出身登记委员会主任 (1997年－1998年)
 - 卢卡斯·巴吉沃西
 - 埃伦·克拉克内尔
 - 英国魔法部
 - 赫克托·弗利
 - 波蒂厄斯·纳奇布尔
 - 奥塔莱恩·甘布尔
 - 霍滕西亚·米里法特
 - 伊万杰琳·奥平顿
 - 魔法部部长 (1849年－1855年)
 - 拉道夫斯·莱斯特兰奇
 - 尤里克·甘普
 - 普里西拉·杜邦
 - 芬里尔·格雷伯克
 - 米里森·巴格诺
 - 禁止滥用魔法办公室助理
 - 杜格德·麦克费尔
 - 骑士公共汽车
 - 英国魔法部部长 (1781年－1789年)
33

社区 8 (标签: 梅拉尼娅·麦克米兰) 包含成员:
 - 布斯巴顿魔法学院
 - 《预言家日报》魁地奇资深记者
 - 亚瑟·韦斯莱
 - 古灵阁巫师银行
 - 禁止滥用麻瓜物品办公室主任
 - 费比安·普威特
 - 弗雷德·韦斯莱
 - 赫敏·格兰杰
 - 罗恩·韦斯莱
 - 赫敏·韦斯莱
 - 拉环
 - 伊格内修斯·普威特
 - 雨果·韦斯莱
 - 路易·韦斯莱
 - 阿不思·波特
 - 在古灵阁巫师银行的兼职
 - 普威特先生
 - 禁止滥用麻瓜物品办公室
 - 韦斯莱魔法把戏坊
 - 阿波琳·德拉库尔
 - 伪劣防御咒及防护用品侦查收缴办公室
 - 韦斯莱家的孩子
 - 莉莉·卢娜·波特
 - 魔法法律执行司副司长
 - 罗丝·韦斯莱
 - 古灵阁巫师银行雇员 (曾经)
 - 哈维·瑞吉比特
 - 霍利黑德哈比队魁地奇球员
 - 格兰杰夫人
 - 莫莉·韦斯莱
 - 波特瞭望站
 - 格兰杰先生
 - 伪劣防御咒及防护用品侦查收缴办公室主任
 - 家养小精灵权益促进会
 - 火龙学家
 - 莫丽·韦斯莱，原姓普威特
 - 穆丽尔
 - 格兰杰家庭


利用AI生成可视化测试过程

In [ ]:

from pyvis.network import Network
import matplotlib.colors as mcolors

def visualize_communities_pyvis(kb, community_labels, output_file="graph_communities.html"):
    """
    使用 PyVis 生成交互式社区可视化图表
    :param kb: Knowledgebase 对象
    :param community_labels: 节点到社区标签的映射字典 {node: label}
    :param output_file: 输出的 HTML 文件名
    """
    # 1. 初始化 PyVis 网络
    # height/width: 画布大小
    # bgcolor/font_color: 背景和字体颜色
    # notebook=True: 在 Jupyter Notebook 中显示
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')
    
    # 2. 准备颜色映射
    unique_labels = list(set(community_labels.values()))
    colors = list(mcolors.TABLEAU_COLORS.values()) # 使用 Matplotlib 的标准色板
    # 循环使用颜色
    label_to_color = {label: colors[i % len(colors)] for i, label in enumerate(unique_labels)}
    
    # 3. 添加节点
    # PyVis 支持批量添加，但为了自定义颜色，我们逐个添加
    all_nodes = kb.get_all_nodes()
    
    # 计算度数用于调整节点大小
    undirected_neighbors = kb.get_undirected_neighbors()
    degrees = {node: len(neighbors) for node, neighbors in undirected_neighbors.items()}
    
    for node in all_nodes:
        community = community_labels.get(node)
        color = label_to_color.get(community, "#ffffff")
        size = degrees.get(node, 1) * 2 + 5 # 基础大小5，每多一个邻居+2
        
        # title 是鼠标悬停时显示的文本
        title = f"Node: {node}<br>Community: {community}<br>Degree: {degrees.get(node, 0)}"
        
        net.add_node(node, label=node, title=title, color=color, size=size)
    
    # 4. 添加边
    # 同样使用无向图逻辑避免重复
    for node, neighbors in undirected_neighbors.items():
        for neighbor in neighbors:
            if node < neighbor:
                net.add_edge(node, neighbor, color="#555555") # 灰色边
    
    # 5. 设置物理引擎参数 (让图布局更自然)
    # 修改：增大阻尼(damping)和斥力(gravity)，解决节点一直震荡碰撞的问题
    net.barnes_hut(
        gravity=-10000,        # 强斥力，让节点散开
        central_gravity=0.3,
        spring_length=200,     # 连线更长
        spring_strength=0.01,  # 弹簧更软
        damping=0.5,           # 高阻尼，快速停止运动
        overlap=1              # 强制避免重叠
    )
    
    # 6. 保存并显示
    print(f"正在生成交互式图谱: {output_file} ...")
    try:
        # 在 Notebook 中显示
        return net.show(output_file)
    except Exception as e:
        print(f"显示失败，请直接打开生成的 HTML 文件: {e}")

# --- 执行可视化 ---
if __name__ == '__main__':
    # 测试louvain算法
    # if 'community_labels' not in locals():
    #     kb = Knowledgebase(DATA_PATH)
    #     community_labels = louvain(kb, max_passes=4)
    
    # 测试LPA算法
    kb = Knowledgebase(DATA_PATH)
    all_nodes = kb.get_all_nodes()
    community_labels = LPA(kb.graph, max_iterations=50,all_nodes = all_nodes)
    # 生成可视化
    visualize_communities_pyvis(kb, community_labels, output_file="harry_potter_communities.html")

Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Batches: 100%|██████████| 45/45 [00:00<00:00, 152.48it/s]


正在生成交互式图谱: harry_potter_communities.html ...
harry_potter_communities.html


### 到此构建知识图谱的社区划分部分就完成了

不管是LPA还是Louvain算法，最终都能得到每个节点所属的社区标签。社区划分的效果具有随机性，和初始条件和迭代顺序有关以及迭代次数有关。
总的来说，感觉实现比较简单，**表现效果没有想象中的那么好**,虽然尝试了不同的做法。

### 图检索
下面图检索主要实现两个功能：
1. 基于贪心搜索算法,区别于普通的BFS，这里通过语义相似度计算，每次选择语意相似度最高的节点进行搜索，实现从起始节点到目标节点的路径检索功能。
2. 基于社区划分结果，实现社区内的图检索功能。

In [123]:
# 图检索 
import heapq
from sklearn.metrics.pairwise import cosine_similarity
# 注意 cosine_similarity 输入需要是二维数组 所以要reshape一下

def retrieve_similar_nodes(kb, start_node, end_node,max_steps= 10):
    '''
    start_node: 起始节点
    end_node: 目标节点
    max_steps: 最大搜索步数
    '''
    if start_node not in kb.nodes or end_node not in kb.nodes:
        print("起始节点或目标节点不存在于图中。")
        return []

    # 获取终点的embedding
    target_emb = kb.embeddings[end_node].reshape(1,-1)

    # 初始化优先队列 heapq 是小顶堆，所以将相似度转为负数，相似度越高的，负数值越小，位于堆顶
    queue = []

    # 起点到终点相似度
    start_emb = kb.embeddings[start_node].reshape(1,-1)
    start_sim = cosine_similarity(start_emb,target_emb)[0][0]
    heapq.heappush(queue,(-start_sim,start_node,[(None,start_node)]))
    visited = set()
    

    # 搜索
    while queue:
        # 弹出当前最相似的节点
        current_score, current_node, path = heapq.heappop(queue)

        if current_node == end_node:
            return path

        if current_node in visited:
            continue

        visited.add(current_node)

        # 剪枝： 超过最大步数就停止这个分支
        if len(path) > max_steps:
            continue

        neighbors = kb.get_neighbors(current_node)

        for relation, neighbor in neighbors:
            if neighbor not in visited:
                # 增加对 None 的检查
                # 某些节点可能没有 embedding (例如孤立点或未正确初始化的节点)
                neighbor_emb_raw = kb.get_embedding(neighbor)
                if neighbor_emb_raw is None:
                    continue
                    
                neighbor_emb = neighbor_emb_raw.reshape(1,-1)
                sim = cosine_similarity(neighbor_emb,target_emb)[0][0]
                new_path = path + [(relation, neighbor)]
                heapq.heappush(queue,(-sim, neighbor, new_path))
    
    return []  # 未找到路径

In [131]:
# 确保前面内容都跑过，后面才能直接用
# --- 测试部分 ---
# 
if __name__ == "__main__":
  
    if 'kb' not in locals():
        kb = Knowledgebase(DATA_PATH)

    # 测试案例：寻找 "哈利·波特" 到 "伏地魔" 的关系
    # 这是一个跨阵营的搜索，普通 BFS 搜索效率低，但语义搜索会优先找"敌人"或"食死徒"相关节点
    start = "哈利·波特"
    end = "蒙太"
    
    path = retrieve_similar_nodes(kb, start, end)
    
    if path:
        print("\n🔍 路径详情:")
        for i, (relation, node) in enumerate(path):
            if relation:
                print(f" --[{relation}]--> {node}")
            else:
                print(f"{node} (起点)")
    else:
        print("未找到路径。")


🔍 路径详情:
哈利·波特 (起点)
 --[~从属]--> 赫敏·格兰杰
 --[~母亲]--> 雨果·韦斯莱
 --[~表弟]--> 詹姆·小天狼星·波特
 --[~后裔]--> 伊格诺图斯·佩弗利尔
 --[哥哥]--> 卡德摩斯·佩弗利尔
 --[~祖先]--> 葛姆蕾·冈特
 --[祖先]--> 萨拉查·斯莱特林
 --[从属]--> 斯莱特林学院
 --[~从属]--> 蒙太


### 利用社区信息优化检索
在图检索过程中，利用社区划分结果可以显著提升检索效率和准确性。具体方法如下：
1. **社区优先搜索**：在进行路径检索时，优先在起始节点所属的社区内进行搜索。这是因为社区内的节点通常具有更高的相关性和连接密度，增加了找到目标节点的概率。
2. **限制搜索范围**：如果目标节点的社区标签已知，可以将搜索范围限制在该社区内，减少不必要的计算和遍历。
3. **动态调整搜索策略**： 如果在社区内未能找到目标节点，可以逐步扩大搜索范围，考虑邻近社区，以平衡搜索效率和准确性。

但是这里通常做法需要提取社区的信息，比如利用LLM生成社区的描述信息，然后计算与目标节点的相似度，再指导搜索过程。这里采取了一种替代的方式，在原来的贪心策略上，将相似度评分修改为：
$$score = \alpha \cdot sim(node, target) + (1 - \alpha) \cdot community\_bonus）（基于社区给予奖励，参数可调整）$$
这样加速我们搜索过程。

In [ ]:
# 利用社区信息优化检索
# 图检索 
import heapq
from sklearn.metrics.pairwise import cosine_similarity

def community_search(kb, start_node, end_node, community_labels, max_steps=20, alpha=0.6):
    '''
    基于社区感知的启发式搜索 (支持重叠社区)
    :param community_labels: 节点到社区标签的映射字典。
                             可以是 {node: label} (LPA/Louvain) 
                             也可以是 {node: {label1, label2}} (SLPA)
    '''
    if start_node not in kb.nodes or end_node not in kb.nodes:
        print("起始节点或目标节点不存在于图中。")
        return []

    # 辅助函数：获取节点的社区集合
    def get_comm_set(node):
        val = community_labels.get(node)
        if val is None:
            return set()
        if isinstance(val, (set, list, tuple)):
            return set(val)
        return {val}

    # 1. 预计算目标信息
    target_emb = kb.embeddings[end_node].reshape(1, -1)
    target_comms = get_comm_set(end_node)
    start_comms = get_comm_set(start_node)
    


    # 2. 初始化优先队列
    queue = []
    visited = {} 

    # 计算起点的初始得分
    start_emb = kb.embeddings[start_node].reshape(1, -1)
    sem_score = cosine_similarity(start_emb, target_emb)[0][0]
    
    # 社区分: 如果有共同社区，则奖励
    has_common_comm = not start_comms.isdisjoint(target_comms)
    comm_score = 1.0 if has_common_comm else 0.0
    
    final_score = alpha * sem_score + (1 - alpha) * comm_score
    
    heapq.heappush(queue, (-final_score, start_node, [(None, start_node)]))
    
    steps_count = 0

    # 3. 搜索循环
    while queue:
        current_score, current_node, path = heapq.heappop(queue)
        steps_count += 1
        
        if current_node == end_node:
            return path

        if current_node in visited and visited[current_node] <= len(path):
            continue
        visited[current_node] = len(path)

        if len(path) > max_steps:
            continue

        neighbors = kb.get_neighbors(current_node)

        for relation, neighbor in neighbors:
            neighbor_emb_raw = kb.get_embedding(neighbor)
            if neighbor_emb_raw is None:
                continue
            
            neighbor_emb = neighbor_emb_raw.reshape(1, -1)
            sim = cosine_similarity(neighbor_emb, target_emb)[0][0]
            
            # B. 计算社区奖励
            neighbor_comms = get_comm_set(neighbor)
            # 只要邻居所在的社区集合 与 目标社区集合 有交集，就给予奖励
            is_in_target_comm = not neighbor_comms.isdisjoint(target_comms)
            
            comm_bonus = 1.0 if is_in_target_comm else 0.0
            
            total_score = alpha * sim + (1 - alpha) * comm_bonus
            
            new_path = path + [(relation, neighbor)]
            
            if neighbor not in visited or len(new_path) < visited[neighbor]:
                heapq.heappush(queue, (-total_score, neighbor, new_path))
    
    return []  # 未找到路径

In [ ]:
# --- 测试部分 ---
if __name__ == "__main__":
   
    if 'kb' not in locals():
        kb = Knowledgebase(DATA_PATH)
    
  
    all_nodes = kb.get_all_nodes()
    # 使用 SLPA 替代 LPA
    community_labels = SLPA(kb.graph, T=200, threshold=0.1, all_nodes=all_nodes)

    start = "哈利·波特"
    end = "蒙太" 
    
    path = community_search(kb, start, end, community_labels, alpha=0.6)
    
    if path:
        print(f"\n✅ 找到路径 (长度 {len(path)-1}):")
        for i, (relation, node) in enumerate(path):
            # 打印社区时处理集合
            comms = community_labels.get(node, set())
            comm_str = ",".join(list(comms)) if comms else "未知"
            
            if relation:
                print(f" --[{relation}]--> {node} (社区: {comm_str})")
            else:
                print(f"{node} (起点, 社区: {comm_str})")

Running SLPA (Overlapping Community Detection)...
Debug: 起点 '哈利·波特' 社区: {'阿不思·西弗勒斯·波特'}
Debug: 终点 '蒙太' 社区: {'蒙太', '贝拉特里克斯·莱斯特兰奇', '霍格沃茨魔法学校'}

✅ 找到路径 (长度 6):
哈利·波特 (起点, 社区: 阿不思·西弗勒斯·波特)
 --[~从属]--> 桃金娘·沃伦 (社区: 桃金娘·沃伦,霍格沃茨魔法学校)
 --[从属]--> 德拉科·马尔福 (社区: 贝拉特里克斯·莱斯特兰奇)
 --[~儿子]--> 卢修斯·马尔福 (社区: 贝拉特里克斯·莱斯特兰奇)
 --[~儿子]--> 阿布拉克萨斯·马尔福 (社区: 贝拉特里克斯·莱斯特兰奇)
 --[从属]--> 斯莱特林学院 (社区: 贝拉特里克斯·莱斯特兰奇)
 --[~从属]--> 蒙太 (社区: 蒙太,贝拉特里克斯·莱斯特兰奇,霍格沃茨魔法学校)
